## Interprete per espressioni aritmetiche

Si consideri la seguente sintassi delle espressioni aritmetiche, estesa con le variabili (`Ide`) e un costrutto sommatoria (`Sumall`)

    Exp ::= N | Ide | Exp op Exp | sumall ( Ide ; n ; n ){Exp}
    op  ::= + | - | * | / 
    N   ::= 0 | 1 | 2 | ...
    Ide ::= x | y | z | ...

La sua sintassi astratta è definita come:

In [2]:
(*Sintassi astratta*)
type op = Add | Sub | Mul | Div ;;
type exp = 
    | Val of int
    | Ide of string
    | Op of op*exp*exp
    | Sumall of string*int*int*exp;; 

type op = Add | Sub | Mul | Div


type exp =
    Val of int
  | Ide of string
  | Op of op * exp * exp
  | Sumall of string * int * int * exp


### Es 1:
Si implementi una funzione `subst (e:exp) (x:ide) (v:exp)` dove `e` è un'espressione, `x` è un identificatore e `v` è un'espressione che deve descrivere un valore (`Val n`). La funzione calcola l'espressione $e\{x:=v\}$ in cui tutte le occorrenze "libere" di `x` in `e` sono sostituite con `v`. 

**ATTENZIONE:** Il costrutto `sumall (x;n1;n2) {e}` lega le occorrenza di `x` in `e`. La sostituzione deve operare solo sulle occorrenze libere della variabile. 

#### Soluzione:
Un'operazione `subst e x v` è una trasformazione **da espressioni a espressioni** (quindi puramente sintattica), che sostituisce tutte e sole le **occorrenze libere** di `x` all'interno di `e` con il valore `v`. Una occorrenza di `x` è libera quando non è legata da un quantificatore, ovvero quando non compare all'interno di una espressione `sumall(x, ..., ...) ...`. 

Questo vuol dire che, se non ci sono occorrenze libere di `x` in `e`, `subst e x v` on ha alcun effetto, e restituisce semplicemente `e`.

Esempi:
- `5 {x:=1}` si formalizza come  `subst(Val(5), "x", Val(1))`, che restituisce `Val(5)`
- `(4+x)*x {x := 2}` si formalizza come `subst(Op(mul, Op(sum, Val(4), Ide("x")), Ide("x")), "x",  Val(2))`, che restituisce `(4+2)*2`. 
- `x {x:=0}` si formalizza come `subst(Ide("x"), "x", Val(0))`restituisce `Val(0)`
- `y {x:=0}` si formalizza come `subst(Ide("y"), "x", Val(0))` e restituisce `Ide(y)`
- `Sumall x che va da 1 a 10 di 5*x {x:=0}` si formalizza come 

    `subst (sumall("x", 1, 10, Op(mul, Val(5), Ide("x")))) "x" Val(0)`
    
   e restituisce `sumall("x", 1, 10, Op(mul, Val(5), Ide("x")))`, perchè `x` è legata.
- `Sumall x che va da 1 a 10 di 5*y {y:=0}` si formalizza come 
    
    `subst (sumall("x", 1, 10, Op(mul, Val(5), Ide("y")))) "y" Val(0)`
   
   e restituisce `sumall("x", 1, 10, Op(mul, Val(5), Val(0)))`



In [18]:
(*subst deve visitare ricorsivamente l'albero, e sostituire 
  le foglie Ide(x) con foglie Val(n), solo quando Ide(x) è libera*)
let rec subst (e:exp) (x:string) (v:exp) = 
    match v with (*per definizione, una sostituzione è possibile solo quando v è un valore*)
    |Val(n) -> 
        (match e with
        |Val(n) -> Val(n)
        |Op(op, e1, e2) -> let e1' = subst e1 x v in
                           let e2' = subst e2 x v in
                           Op(op, e1', e2')
        |Ide(id) -> if id = x then Val(n)
                   else Ide(id)
        |Sumall(id, n1, n2, _e) -> if id = x 
                                    (*se id = x, allora tutte le occorrenze di x in _e sono
                                      legate, quindi possiamo fermare la visita ricorsiva.*)
                                    then Sumall(id, n1, n2, _e)
                                    (*altrimenti, ci chiamamo ricorsivamente su _e*)
                                   else Sumall(id, n1, n2, (subst _e x v))
        )
    |_ -> failwith "errore di sostituzione";;
    
    

val subst : exp -> string -> exp -> exp = <fun>


**TEST:** applicare la funzione `subst` in modo da calcolare $((x+y)+(\mbox{sumall }x;1;10) \, x) \{x:=5\}$ che deve restituire $((5+y)+(\mbox{sumall }x;1;10)\, x)$.

In [5]:
let e1 = Op (Add, (Op (Add, Ide "x", Ide "y")), (Sumall ("x",1,10,Ide "x"))) ;;
subst e1 "x" (Val 5);;

val e1 : exp =
  Op (Add, Op (Add, Ide "x", Ide "y"), Sumall ("x", 1, 10, Ide "x"))


- : exp = Op (Add, Op (Add, Val 5, Ide "y"), Sumall ("x", 1, 10, Ide "x"))


### Es 2:

Si modifichi il codice (riportato qui sotto) dell'interprete big step delle espressioni in modo tenere conto della semantica estesa con il costrutto `sumall`, definita come segue. Per l'operazione di sostituzione $e \{x := v\}$ utilizzare la funzione `subst` definita nell'esercizio precedente.

$$
n \rightarrow_{bs} n
\qquad
\frac{E_1 \rightarrow_{bs} n_1 \quad E_2 \rightarrow_{bs} n_2 \quad n_1 \mbox{op } n_2 = n}{E_1 \, op \, E_1 \rightarrow_{bs} n}
$$

$$
\frac{e\{x:=n\} \rightarrow_{bs} n'}
{\mbox{sumall } ( x ; n ; n ) \, e \rightarrow_{bs} n'}
$$

$$
\frac{n_1<n_2 \quad e\{x:=n_1\} \rightarrow_{bs} n' \quad \mbox{sumall } ( x ; n_1+1 ; n_2 ) \, e \rightarrow_{bs} n'' \quad n = n' + n''}
{\mbox{sumall } ( x ; n_1 ; n_2 ) \, e \rightarrow_{bs} n}
$$

**NOTA1:** non sono previste regole per gli identificatori (sono gestiti dalla sostituzione).

**NOTA2:** la semantica non prevede transizioni in caso di identificatori liberi (non legati da `sumall`) o indici numerici usati nel costrutto `sumall` non ben definiti

#### Soluzione:

La funzione `eval` si ricava passo passo dalle regole di inferenza, tenendo conto che:
-  $n$ sono i valori, quindi `Val(...)`
-  $e\{x := n\}$ è una sostituzione, quindi `subst e x n`
- $\rightarrow_{bs}$ denota il passaggio da una espressione al suo valore semantico big step, quindi è una chiamata della funzione `eval`.

È importante notare che l'interprete procede per ricorsione strutturale, cioè il risultato di `eval e` dipende dal risultato di `eval e'`, dove `e'` è una sottoespressione di `e`, e quindi è **sintatticamente più semplice** (cioè composta da meno operatori, a.k.a. `e'` è un AST con profondità minore). L'unica eccezione a questo approccio è l'ultima regola di inferenza, corrispondente al ramo else nella riga 18. Qui vanno effettuate due chiamate ricorsive: una su `_e`, che è sintatticamente più semplice di `e`, e l'altra sull'espressione `Sumall(id, n1+1, n2, _e)`, che ha la stessa complessità sintattica, ma è semanticamente un passo più vicino alla soluzione, perchè si avvicina al caso base `Sumall(id, n2, n2, _e)`.

In [6]:
(* interprete big-step *)
let rec eval e =
    match e with
    | Val n -> Val n
    | Op (op,e1,e2) -> 
        (* chiamate ricorsive che calcolano le derivazioni per e1 ed e2 *)
        (match (eval e1, eval e2) with
        | (Val n1, Val n2) -> (match op with   (* calcola n1 op n2 *)
                               | Add -> Val (n1+n2)
                               | Sub -> Val (n1-n2)
                               | Mul -> Val (n1*n2)
                               | Div -> Val (n1/n2)
                               )
        (* caso (inutile) aggiunto solo per rendere esaustivo il pattern matching *)
        | _ -> failwith "Errore impossibile che si verifichi" )
    | Sumall(id, n1, n2, _e) -> if n1 = n2 then 
                                        eval (subst _e id (Val(n1)))
                                else if n1 < n2 then 
                                        let e1 = eval (subst _e id (Val(n1))) in 
                                        let e2 = eval (Sumall(id, n1+1, n2, _e)) in
                                        match e1, e2 with
                                        |Val(n1), Val(n2) -> Val(n1+n2)
                                        |_ -> failwith "Errore impossibile che si verifichi"
                                else failwith("Errore: sumall mal formulato")
    | Ide _ -> failwith("Errore: identificatore libero");;

val eval : exp -> exp = <fun>


**TEST:** calcolare il risultato delle seguenti espressioni:

* sumall (x; 1; 10) x
* 100 + sumall (x;1;10) (sumall (y;1;10) x*y)

Risultati attesi: 55 e 3125

In [7]:
let e2 = Sumall ("x",1,10,Ide "x");;
let e3 = Op (Add, Val 100, 
                 (Sumall ("x",1,10,(Sumall ("y",1,10, Op ( Mul, Ide "x", Ide "y")))))) ;;
                 
eval e2;;
eval e3;;

val e2 : exp = Sumall ("x", 1, 10, Ide "x")


val e3 : exp =
  Op (Add, Val 100,
   Sumall ("x", 1, 10, Sumall ("y", 1, 10, Op (Mul, Ide "x", Ide "y"))))


- : exp = Val 55


- : exp = Val 3125
